In [ ]:
import numpy as np
import plotly.express as px
def plot_embedding(embedding, labels=None):
    """
    Plot a 2D embedding with optional labels for coloring.

    Args:
        embedding (ndarray): shape (n_samples, 2)
        labels (array-like, optional): labels for coloring points
    """
    if labels is None:
        labels = np.zeros(embedding.shape[0])  # default: all same


    fig = px.scatter(
        x=embedding[:, 0],
        y=embedding[:, 1],
        color=labels.astype(str),  # convert to string for discrete legend
        title="2D Embedding Visualization",
        labels={"x": "Dim 1", "y": "Dim 2", "color": "Label"}
    )


    fig.update_layout(
        width=1000,
        height=800,
        title="Custom size Plotly figure"
    )


    fig.show()



In [ ]:
from sklearn.manifold import TSNE

def run_tsne(X_vec, perplexity=30, n_iter=1000, learning_rate=200, random_state=42):
    """
    Run t-SNE on data X and return the 2D embedding.

    Parameters
    ----------
    X : array-like, shape (n_samples, n_features)
        Input data.
    perplexity : float, optional
        Balances local/global structure (default: 30).
    n_iter : int, optional
        Number of optimization iterations (default: 1000).
    learning_rate : float, optional
        Learning rate for t-SNE (default: 200).
    random_state : int, optional
        Random seed for reproducibility (default: 42).

    Returns
    -------
    X_embedded : ndarray, shape (n_samples, 2)
        The t-SNE embedding of the data.
    """
    tsne = TSNE(
        n_components=2,
        perplexity=perplexity,
        learning_rate=learning_rate,
        n_iter=n_iter,
        random_state=random_state
    )
    X_embedded = tsne.fit_transform(X_vec)
    return X_embedded


In [ ]:
import scanpy as sc
nk_path='/Users/chandrasekharmukherjee/PycharmProjects/Personal/cplearn_biotest/NK_cells.h5ad'
adata = sc.read_h5ad(nk_path)

print("done")

X=adata.obsm['X_HARMONY']
label=adata.obs['ident'].to_numpy()
from scipy.sparse import issparse
X_umap=adata.obsm['X_UMAP']

X_cg = adata.X.toarray() if issparse(adata.X) else adata.X
gene_names = adata.var_names.values

print(X_cg.shape,gene_names.shape,gene_names[0:5])

In [ ]:
f_path='/Users/chandrasekharmukherjee/Data/scRNA_IBD/'
d_path=['bcells_final.h5ad',
'cd4tcells_final.h5ad',
'cd8tcells_final.h5ad',
'epicolonic_final.h5ad',
'fibperi_final.h5ad',
'ilc_final.h5ad',
'ilealepi_final.h5ad',
'myeloid_final.h5ad',
'plasmacells_final.h5ad']

subset_number=1

import scanpy as sc
adata=sc.read_h5ad(f_path+d_path[subset_number])
X=adata.obsm['X_harmony']
n=X.shape[0]
print(n)
label=adata.obs['final_analysis'].to_numpy()
pt_label=adata.obs['Patient'].to_numpy()

In [ ]:
import data_call
import numpy as np
d_name='Muraro'
X,label=data_call.get_dataset(d_name, mode='PCA')

In [ ]:
from cplearn_v4.corespect import CorespectModel
from cplearn_v4.corespect.config import (
    FlowRankConfig,
    StableCoreConfig,
FineGrainedConfig,
ClusterConfig,
PropagationConfig
)

Overall parameters:

q,r for FlowRank.

core_frac for stable_core and finegrained core. (There is also an auto-select option that needs more testing)

Resolution for stable core and fine-grained core (for the filtration step).

Densification stable core, then twice in fine-grained core. Also in clustering.

Resolution in clustering.
There is also auto select resolution step in clustering (Not sufficiently tested).

Making all three resolution can have some unexpected effects. Can I auto-calibrate based on initial resolution?

Then propagation has normalized, connectivity, momentum/default, alpha




In [ ]:
flow_cfg = FlowRankConfig(q=20, r=10)
stable_cfg = StableCoreConfig(core_frac=0.2, resolution=2 ,ng_num=10, densification=False)
fine_cfg= FineGrainedConfig(fine_grain_densification=False,core_frac=0.2)
cluster_cfg=ClusterConfig(resolution=2.25,auto_select_resolution=False,densification=False)
prop_cfg=PropagationConfig(normalized=True,connectivity='sym',mode='momentum',alpha=0.2)
model = CorespectModel(
    X,
    flowrank_cfg=flow_cfg,
    stable_cfg=stable_cfg,
    fine_core_cfg=fine_cfg,
    cluster_cfg=cluster_cfg,
    prop_cfg=prop_cfg
).run(fine_grained=True)

print("Running CoreSPECT\n")

In [ ]:
from collections import Counter
print(Counter(model.labels_))
print(Counter(label))
print([len(layer) for layer in model.layers_])
print(X.shape[0])

In [ ]:
import umap
reducer=umap.UMAP()
X_umap=reducer.fit_transform(X)

In [ ]:
m_labels=model.labels_
clusters=[[] for _ in range(len(set(m_labels)))]
for i in model.layers_[0]:
    clusters[m_labels[i]].append(i)

In [ ]:
from cplearn_v4.coremap import Coremap
cmap=Coremap(model,global_umap=X_umap,fast_view=True,anchor_finding_mode='default')
from cplearn_v4.coremap.vizualizer import visualize_coremap
fig=visualize_coremap(cmap,model.labels_, use_webgl=True)
fig.show()

In [ ]:
import umap
reducer=umap.UMAP()

print(model.prob_matrix_.shape)

X_fmap=reducer.fit_transform(model.prob_matrix_)
X_tsne=run_tsne(model.prob_matrix_)


In [ ]:
from cplearn_v4.coremap import Coremap
cmap=Coremap(model,global_umap=X_tsne,fast_view=True,anchor_finding_mode='default')
from cplearn_v4.coremap.vizualizer import visualize_coremap
fig=visualize_coremap(cmap,model.labels_, use_webgl=True)
fig.show()

In [ ]:
from cplearn_v4.coremap import Coremap
cmap=Coremap(model,global_umap=X_fmap,fast_view=True,anchor_finding_mode='default')
from cplearn_v4.coremap.vizualizer import visualize_coremap
fig=visualize_coremap(cmap,model.labels_, use_webgl=True)
fig.show()

In [ ]:
model.update_config("cluster",resolution=1,auto_select_resolution=False,densification=False)
model.cluster_core()
model.update_config("propagation",mode='momentum',connectivity='out',normalized=True,alpha=0.1)
model.propagation_from_core()

In [ ]:
fig=visualize_coremap(cmap,label, use_webgl=True)
fig.show()

In [ ]:
import umap
reducer=umap.UMAP()
X_umap=reducer.fit_transform(X)

In [ ]:
plot_embedding(X_umap,label)

In [ ]:
from cplearn_v4.coremap import Coremap
cmap=Coremap(model,global_umap=X_umap,fast_view=False,anchor_finding_mode='default')
from cplearn_v4.coremap.vizualizer import visualize_coremap
fig=visualize_coremap(cmap,model.labels_, use_webgl=True)
fig.show()

In [ ]:
fig=visualize_coremap(cmap,label, use_webgl=True)
fig.show()

In [ ]:
fig.write_html('CD4+_cplearn_new_providedlabel.html')

In [ ]:
print([len(layer) for layer in model.layers_])

In [ ]:
print(len(model.layers_[0]))
plot_embedding(X_umap[model.layers_[0]],label[model.layers_[0]])
plot_embedding(X_umap[model.layers_[1]],label[model.layers_[1]])

In [ ]:
plot_embedding(X_umap[model.layers_[0]],model.labels_[model.layers_[0]])

In [ ]:
import umap
reducer=umap.UMAP(n_neighbors=1000)
X_umap_core=reducer.fit_transform(X[model.layers_[0]])


In [ ]:
X_F=model.prob_matrix_

In [ ]:
X_tsne=run_tsne(X_F,perplexity=200,n_iter=500,learning_rate=200)

In [ ]:
for layer in model.layers_:
    plot_embedding(X_tsne[layer],model.labels_[layer])

In [ ]:
plot_embedding(X_tsne,model.labels_)